In [721]:
from openai import OpenAI
client = OpenAI(api_key="any-key", base_url="http://mn-dgx01.x5.ru:8000/v1")

In [761]:
def get_text_with_links(document):
    result_text = ""
    
    for page_num in range(len(document)):
        page = document.load_page(page_num)

        # Получаем слова и их координаты
        words = page.get_text("words")
        links = page.get_links()

        # Временный словарь для отслеживания последнего индекса слова для каждого URL
        last_occurrence_with_url = {}

        wc_map = {tuple(word[:4]): word[4] for word in words}
        
        for link in links:
            link_rect = fitz.Rect(link["from"])
            uri = link.get('uri', 'Нет ссылки')
            
            for i, word in enumerate(words):
                text = word[4]   # текст слова
                word_rect = fitz.Rect(word[:4])

                # Проверка на пересечение координат
                if word_rect.intersects(link_rect):
                    last_occurrence_with_url[uri] = (i, tuple(word[:4]))

        # Заменяем текст в конечном словаре ссылками на последних вхождениях
        for uri, (index, rect) in last_occurrence_with_url.items():
            wc_map[rect] = f'{wc_map[rect]}:"{uri}"'

        # Сортировка слов по их координатам для правильного порядка
        sorted_words = sorted(wc_map.items(), key=lambda x: (x[0][1], x[0][0]))

        for item in sorted_words:
            result_text += item[1] + " "
    
    return result_text.strip()  # добавляем strip для удаления лишних пробелов в конце

In [810]:
def get_ner_reply(messages, print_it=False):
    reply = client.chat.completions.create(
        messages=messages,
        model="x5-airun-medium",
        seed=42,
        #top_k=50,
        top_p=0.95,
        temperature=0.3,
        max_tokens=600,
        frequency_penalty=0.5,
        presence_penalty=-0.5,
    ).choices[0].message.content.replace('null', 'None').replace('\\"', '').replace('\n', '').replace('""', '"').replace('": "}', '": ""}').replace(' ",', ' "",').replace(' "]', ' ""]').replace('": "  }', '": ""}').replace('} }', '}}')
    if print_it:
        print(reply)
    try:
        try:
            try:
                dict_reply = ast.literal_eval(reply[reply.find('{'):reply.rfind('}')+1])
            except:
                dict_reply = ast.literal_eval(reply[reply.find('{'):reply.find('}}')+2])
        except:
            if "Другие ссылки" in reply:
                dict_reply = ast.literal_eval(reply[reply.find('{'):reply.rfind("Другие ссылки")-1]+'}}')
            elif "Другие контакты" in reply:
                dict_reply = ast.literal_eval(reply[reply.find('{'):reply.rfind("Другие контакты")-1]+'}}')
            else:
                #reply = reply.replace('\n', '')
                dict_reply = ast.literal_eval(reply[reply.find('{'):reply.rfind('}')+2])
                for i in list(dict_reply['Контакты'].keys()):
                    if i not in ['Телефон', 'Telegram', 'Email', 'LinkedIn', 'GitHub']:
                        del dict_reply['Контакты'][i]
                        
        new_reply = deepcopy(dict_reply)

        for k, v in dict_reply.items():
            if isinstance(v, dict):
                new_subdict = {}
                for vk, vv in v.items():
                    if vv is not None and vv != "":
                        if isinstance(vv, list):
                            cleaned_list = [i for i in vv if i is not None and i != ""]
                            if cleaned_list:
                                new_subdict[vk] = cleaned_list if len(cleaned_list) > 1 else cleaned_list[0]
                        else:
                            new_subdict[vk] = vv
                new_reply[k] = new_subdict
        new_reply = {k: v for k, v in new_reply.items() if v or v == 0}
        
        return json.dumps(new_reply, ensure_ascii=False)
    except:
        print('not json')
        return reply

In [723]:
from glob import glob
import pymupdf
from tqdm import tqdm
import ast, json
from copy import deepcopy

In [726]:
paths = glob('../interns_preprocessed/interns_preprocessed/*/*.pdf')

In [805]:
ner_prompt = """Извлеки из резюме ФИО и контакты, рассортируй ссылки в формате json. Структура ответа:
{"ФИО": "", "Контакты": {"Телефон": "", ...}}

В ответе могут быть Телефон, Telegram, Email, LinkedIn, GitHub и другие контакты.
Если какая-то информация в тексте и ссылках не указана, пиши null.
Ссылки всегда пиши в "" по краям.
Указывай полные ссылки с http/https если возможно.
Российские номера начинаются с 8 либо с +7.
Обязательно закрывай списки и json.
Резюме: """

In [ ]:
with pymupdf.open(paths[291]) as doc:  # open document
    text = get_text_with_links(doc)
messages = [
    {"role": "user", "content": ner_prompt+text}
]
print(get_ner_reply(messages, print_it=True))

In [794]:
# with pymupdf.open(paths[90]) as doc:  # open document
#     text = chr(12).join([page.get_text() for page in doc])
#     text = 'Ссылки:\n'+'\n'.join(set([uri['uri'].replace('mailto:','') if 'uri' in uri 
#                                       else uri['file'].replace('mailto:','') if 'file' in uri
#                                       else ''
#                                       for page in doc for uri in page.get_links()]))+'\nТекст резюме:\n'+text
# messages = [
#     {"role": "user", "content": ner_prompt+text}
# ]
# print(get_ner_reply(messages, print_it=False))

In [754]:
# print(text)

In [696]:
# print(df.loc[178].text)

In [815]:
pdf_extract = []
for pdf_file in tqdm(paths):
    with pymupdf.open(pdf_file) as doc:  # open document
        # text = chr(12).join([page.get_text() for page in doc])
        # text = 'Ссылки:\n'+'\n'.join(set([uri['uri'].replace('mailto:','') if 'uri' in uri 
        #                               else uri['file'].replace('mailto:','') if 'file' in uri
        #                               else ''
        #                               for page in doc for uri in page.get_links()]))+'\nТекст резюме:\n'+text
        text = get_text_with_links(doc)
    messages = [
        {"role": "user", "content": ner_prompt+text}
    ]
    pdf_extract.append([pdf_file, text, get_ner_reply(messages)])

100%|█████████████████████████████████████████| 311/311 [15:23<00:00,  2.97s/it]


In [816]:
import pandas as pd

In [817]:
df = pd.DataFrame(pdf_extract, columns=['filepath', 'text', 'NER'])

In [818]:
df.loc[0].text != df.loc[1].text

True

In [819]:
df.to_csv('../ner_pdf.csv', index=False)

In [ ]:
for i in range(311):
    print(i, df.NER.loc[i])

In [685]:
# print(df.loc[97].text)